In [ ]:
import os
import warnings
import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import torch
from models.IntensityForecasting import LPSIntensityForecasting

In [ ]:
data = pd.read_csv("processed/all-lps-dataframe.csv").drop(columns=['Unnamed: 0'])
bg_data = pd.read_csv("processed/data_with_bg.csv").drop(columns=['Unnamed: 0']).drop(columns=['Unnamed: 0.1'])

data['Genesis_Date'] = pd.to_datetime(data['Genesis_Date'])
data['DateTime'] = pd.to_datetime(data['DateTime'])
data["Q850_bg"] = bg_data["Q850_bg"]
data["VS_bg"] = bg_data["VS_bg"]

time_idx = []
for i in range(1,len(data.groupby("id").count())+1):
  for j in range(0,data.groupby("id").count()["Genesis_Date"][i]):
    time_idx.append(j)
data["time_idx"] = time_idx

rolling_columns = ['Latitude', 'Longitude', 'mslp', 'ls_ratio',
       'VO550', 'VO750', 'VO850', 'PV', 'Q850', 'Q850_grad', 'Q2', 'US_850',
       'UN_850', 'VE_850', 'VW_850', 'T2', 'Z_tilt', 'integrated_mse', 'Z250',
       'Z550', 'Z850', 'RF']  
window_size = 6
data_rolling = data.groupby('id')[rolling_columns].rolling(window=window_size, min_periods=1).mean().reset_index(drop=True)
data[rolling_columns] = data_rolling[rolling_columns]

In [ ]:
# Define the max allowed rows (5 days)
max_rows = 8 * 24 

def process_track(track):
    track_length = len(track)
    
    # If track is longer than 5 days, crop it
    if track_length > max_rows:
        track = track.iloc[:max_rows]
    
    return track

# Apply function to each track
data = (
    data.groupby('id', group_keys=False)
    .apply(process_track)
)

# Reset index
data.reset_index(drop=True, inplace=True)

In [ ]:
unknown_variables = ["Latitude", "Longitude", "mslp", 'ls_ratio', 'VO850', "PV", "T2", "Q850",
                     "Q2", "UN_850", "US_850", "VE_850", "RF" ]
max_prediction_length = 5*24
max_encoder_length = 24
bg_data = ["Q850_bg", "VS_bg"]
target = "mslp"

intensity_tft_model = LPSIntensityForecasting(
    data=data, target=target, max_prediction_length=max_prediction_length,
    max_encoder_length = max_encoder_length,
    unknown_variables = unknown_variables,
    bg_data = bg_data
    )

In [ ]:
intensity_tft_model.train()

In [ ]:
intensity_tft_model.evaluate()